# 2.1 리플 데이터 NLP
> 목적 : 키워드별 리플 csv파일을 이용해 각 월별 빈도수를 xlsx로 export한다.   
    

> * 추가 task 1.    )   
: 수집된 전체 리플 중에 '국민'이 들어간 리플만 뽑아내어 csv 편성하기  


> 작업 : "" pos태깅 + 클렌징 ""   
=> task1).  Konlpy를 써서 pos태깅 > '조사','어미','문장부호'등을 drop하기
=> task2).너무 쪼개어진 단어 붙여주기 > 예) '김''무성' > '김무성'    
=> task3).숫자, 한 음절 drop하기     
=> task4).불용어(stopwords) 설정하기      
=> task5).최빈단어순으로 정리된 데이터프레임을 csv파일로 export하기    
=> task6).최빈단어 랭크에 들지 못해도 꼭 넣어주어야하는 특정단어 지정하여 최빈 단어 데이터 프레임에 넣어주기   
=>     

> * 추가 task 2)   
concordance 체크 :     
nltk.text.concordance('원하는 단어') 명령을 통해서, 원하는 단어가 수집한 리플들 가운데 어떻게 쓰이고 있는지를 확인할 수 있게 해줍니다. 

#### * 필요 패키지 import

In [1]:
import pandas as pd
import numpy as np
import datetime  # < 'datetime' 데이터 형태를 조작하기 편하도록 해줍니다. * 참고 pandas.datetime

import platform
import matplotlib.pyplot as plt

import re     #파이썬에서 정규표현식(Regular Expression) 지원 모듈

%matplotlib inline 
# < matpliotlib 을 주피터 노트북에서 사용할 경우 plot들을 결과 창 안에 맞춰 출력시켜주도록 해줍니다. 
# 설정하지 않아도 크게 문제가 되지는 않지만, matpliot을 쓸 때 custumary하게 주로 지정해주고 시작합니다. 

# matplot 한글 폰트 인식 코드 
# matpliot 자체에서는 한글폰트를 자동으로 인식하지 못합니다. 
# matpliot의 plotting paremeter에서 직접 한글폰트 경로를 넣어주면 plot에 한글 출력이 가능해집니다. 
path = "c:/Windows/Fonts/malgun.ttf" # < 윈도우에 '맑은 고딕' 폰트의 경로입니다.
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':  # < 맥킨토시 사용자의 경우 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # < 윈도우 사용자의 경우 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name)
    # 사용하고자 하는 한글 폰트의 경로를 path에 지정해주세요.
else:
    print('Unknown OS')    

plt.rcParams['axes.unicode_minus'] = False

* nltk (Natural Language Toolkit) < 기본적인 자연어 처리 작업을 도와주는 패키지입니다.    
=> 한글에 맞춰진 패키지가 아니기 때문에, 한글 NLP 과정에는 '토크나이즈' 작업을 nltk로 할 수 없지만     
'최빈 단어','연어(collocation)','단어 색인(concordance)','산포도 그림(dispersion_plot)'등을 쓰기 좋습니다. 
(참고: https://datascienceschool.net/view-notebook/8895b16a141749a9bb381007d52721c1/)
(참고: (nltk class 공식페이지) :https://www.nltk.org/_modules/nltk/text.html)

* Konlpy (Python package for Korean natural language processing)    
(코엔엘파이 공식 페이지 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/)   
한국어 자연어 정보처리를 위한 패키지입니다. 한글 문법에 따라 형태소 분해, 토크나이징을 수행할 수 있게 해줍니다. Konlpy 안에 Hannanum Class(한나눔), Kkma Class(꼬꼬마), Komoran Class(코모란), Mecab Class(미캅), Twitter Class(트위터) 클래스들이 존재합니다.   
( 참고 : https://datascienceschool.net/view-notebook/a0237ff8f13a454c96072f868c01bc30/)   

각각의 클래스들은 '자연어'를 받아서 국어 문법에 맞게 분해하는 기준이 조금씩 다르고, 그에 따라 처리속도의 차이,  pos태깅 차이(pos : part of speech, 품사) 등이 장단점이 각각 다릅니다.    
( 클래스 간 차이 참고 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

In [2]:
import nltk

from konlpy.tag import Okt # < twitter 클래스가 Okt로 이름이 바뀌었습니다.
#Twitter 로 사용은 아직 가능하나 (deprecated) 예정입니다. 
okt = Okt()

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from wordcloud import ImageColorGenerator

C:\Users\seoun\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


## 언론3사 + 전체기간 + 공천.csv import 하기

In [3]:
total = pd.read_csv('../리플통합/공천_모든리플.csv', engine='python', header=0, index_col = None)


In [4]:
total['rp_date'] = total.rp_date.astype('str') # < csv를 불러오면, 날짜의 dtype을 소수(float)로 인식해서 
# '2019.10' 값이 > 2019.1 로 바뀌어 버리는 문제가 있습니다. 
# 그래서 날짜열을 다시 문자(string)로 바꾼 후에 
total.loc[total['rp_date'] == '2019.1', 'rp_date'] = '2019.10'  # < 2019.1값을 '2019.10'으로 다시 바꿔줍니다. 

# 리플 항목에 NaN 이 되어 있는 경우, pos태깅에 에러가 발생합니다. 
# 따라서 결측값은 '띄어쓰기',혹은 '(아무것도 없는 칸)' 으로 만들어줍니다. 
total.fillna(' ',inplace = True)
total.reset_index(drop = True, inplace = True)

In [5]:
# 공천 키워드에 담긴 댓글들의 날짜를 확인합니다. 
every_months = total['rp_date'].unique().tolist()
every_months
# 이 월별 값을 for 반복문으로 돌려
# 월별 최빈 단어 50개를 우선 csv파일로 export하는게 목적입니다. 

['2016.01', '2016.02', '2016.03', '2019.10', '2019.11', '2019.12']

## 공천 _ 빈도 _ 월별.xlsx 제작

### * 필터링 변수 설정 

In [6]:
stop_words = ['하다', '다', '들', '이', '안', 'ㅋㅋ','것', '적','ㅎㅎ','40%','대다', # << stop_words 1차 : 
                  'ㅋㅋㅋ', '그', '거', '이다', '더', '수', '건', '지', '내', '요', 'ㅋ', '한', '이번',
                  '임', '저', '하', '근데', '다른', '결국', '제', '겠다', '어디', '나', '지다', '석', '공', '뿐',
                  '게', '답', '때문', '웃기', '먼저', 'ㄷㄷㄷ', '성', '원래', '쪽', '이면', '뭐', '또', '올', '그게',
                  '저런', '3', '대', '위해', '반', '가장', '그렇게', '비', '질', '그래서', '보', '날', '누구', '상',
                  '보다','되다', '있다', '없다', '아니다', '같다', # << stop_words 2차 : 서술어
                  '들다', '그렇다','안되다','않다','가다','못','나오다',
                  "관리자", "삭제하였습니", "○○○", "너희들", "하는건", "추한인간떨어져서", 
                  "안봤으면","이야기","정부용비어천가",
                  '받다', '많다', '시키다', '보내다', '되어다', '들이다', '이제', '버리다', '만들다', '살다', 
                  '차다', '쓰다', '맞다', '말다', '자기', '하의', '그리고', '당신', '이제', '나다',
                  '하나', '너무', '이렇다', '넘다' ] # << 불용어 3차
# 불용어에는 다시 넣어다 뺐다 핼 필요가 없을 정도로 불필요한 토큰만 넣어준다. 
# 백업용 빈도 csv에 많은 토큰을 넣어서 export하고 >
# 시각화 단계에서 쓸/안 쓸 단어 2차 필터링 가능하니까 

# 꼭 포함해주어야 하는 필수 단어를 이 안에 넣는다!
necessary_words=['동물','짐승','문희상','아들','세습','논란','불출마','황교안','친황','청와대','문재인',
                 '식물','배신', '배제']

# 빈도가 높은 단어 '몇 등' 까지 보관할지 정해줍니다.
freq_rank_upto = 70

## 추가 task 1.
* '핵심단어'가 들어있는 리플에 대해서만 새로 csv를 뽑고자 할 때

In [ ]:
Target_keyword = '공천'
core1_keyword = '국민' # 추가 task 1) : 리플안에 '국민'이 들어간 경우 따로 빼어 새 csv에 보냅니다. 

word_Yes_idx = [] # < '국민'이 있는 리플의 index번호를 담을 리스트

total_lines = total['rp_text']

total_lines = list(map(lambda x : str(x).replace('\r','').replace('\n','').replace('@','').replace('*','') , total_lines))   

print('3사 댓글 갯수 : ',len(total_lines))

# 1차 분류를 넣을 리스트
results = []
lines = total_lines
# 돌아가지 않은 리스트를 넣을 리스트
error_lines = []
line_idx = 0
for line in lines: # 리플 한 줄씩 내용을 받아서
    try:
        malist = okt.morphs(line) # 매 문장 마다 각 형태소 분해
        if core1_keyword in malist:
            word_core1_idx.append(line_idx)
    except:
        error_lines.append(line)
    line_idx += 1
    if (line_idx % 10000) ==0 :
        print(str(line_idx),'번째 라인')
        
file_name_core1 = '../' + Target_keyword + '_+' + core1_keyword + '_리플.csv'

save_df_all = total.loc[word_Yes_idx,]
save_df_all.reset_index(drop = True, inplace = True)
save_df_all.to_csv(file_name_core1, encoding = 'EUC-KR', index = False)
print('고유 댓글 총 : ',len(total),'개 중에')
print('리플에 ' ,core1_keyword, ' 이 등장한 댓글 수 :', len(save_df_all), '개')
print(file_name_core1, '  --->  저장 완료')


## 리플 데이터 NLP 작업 수행

In [23]:
Target_keyword = '공천'

for each_month in every_months: # 위에서 나온 월별 값을 하나씩 받습니다. # ['2016.01', '2016.02', .. ,'2019.12']

    print(Target_keyword, str(each_month), ' 빈도 분석 수행\n')
    total_lines = total.loc[total.rp_date == each_month,'rp_text']

    total_lines.reset_index(drop = True, inplace = True)
    total_lines = list(map(lambda x : str(x).replace('\r','').replace('\n','').replace('@','').replace('*','') , total_lines))   

    print('3사 댓글 갯수 : ',len(total_lines))

    # ========================== 월별 NLP 수행 ==========================
    # '공천'에 대한 빅데이터 분석 
    
    # 1차 분류를 넣을 리스트
    results = []
    lines = total_lines
    # 돌아가지 않은 리스트를 넣을 리스트
    error_lines = []

    print(' \n task1 : 불필요한 품사 제거     ===== \n')
    print('리플 수 : ',len(lines))

    line_idx = 0
    for line in lines: # line = 리플 하나(ex:'둘 다 죽어줘야 국민들이 행복... 중략... ㅎㅎㅎ')씩 받아서
        try:
            malist = okt.pos(line, norm=True, stem=True) # 매 문장 마다 pos tagging하면...
            # malist = [ ('죽다', 'Verb'),('국민', 'Noun'),...] 이런 형태가 되고
            r= []                 
            for word in malist:
                if not word[1] in ["Josa", "Eomi", "Punctuation"]: # 조사,어미,문장부호를 뺀 나머지만 담아서
                    r.append(word[0])

            r1 = (" ".join(r)).strip()
            results.append(r1)
        except:
            error_lines.append(line)
        # 작업 진행 정도를 보기위해 50,000번째 리플마다 print
        line_idx += 1
        if (line_idx % 10000) ==0 :
            print(str(line_idx),'번째 리플까지   ---->   pos 필터링')

    print('pos태깅 에러 라인 수: ', len(error_lines) )    
    print('조사, 어미, 문장부호 제거 후')
    print('ex) 0번째 리플 : ',lines[0],'\n     ====> pos 필터링 후 =>', results[0])


    # 조사, 어미, 문장부호를 뺀 리스트 요소를 > 하나의 text 객체로 만든다
    WholeText = "".join(results) # result원소 하나하나로 쪼개져 있는 리플들을 모두 join 합니다. 
    print('\n     ====> WholeText[0:20]... : ',WholeText[:20])

    # 다시 형태소 단위로만 만들어서
    first_tokens = okt.morphs(WholeText)
    print('\n 1. Tokenization : \n   토큰 갯수 =====>',len(first_tokens))
    print('ex) : first_tokens[:10]... => ',first_tokens[:10])

    ### > 2차 클랜징
    print('\n 2. 클렌징 : task2+3. 해부된 단어 재결합 + 한 글자, 숫자 filtering-out')
    # task2. De-dissectification >>> 너무 쪼개진 단어를 강제로 복원시킨다.
    line_idx = 0
    for i in range(len(first_tokens)):
        len(first_tokens) / 
        if i != int(len(first_tokens) -1):
            if (first_tokens[i] == '창')&(first_tokens[i+1] == '당하다')  :
                first_tokens[i] = '창당'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '소')&(first_tokens[i+1] == '리지')  :
                first_tokens[i] = '소리'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '한국')&(first_tokens[i+1] == '당')  :
                first_tokens[i] = '한국당'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '친')&(first_tokens[i+1] == '문')  :
                first_tokens[i] = '친문'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '지')&(first_tokens[i+1] == '지층')  :
                first_tokens[i] = '지지층'
                first_tokens[i+1] = ' '
            elif first_tokens[i] == '태경':
                first_tokens[i] = '하태경'
            elif first_tokens[i] == '유투브':
                first_tokens[i] = '유튜브'
            elif (first_tokens[i] == '자')&(first_tokens[i+1] == '한')  :
                first_tokens[i] = '자한당'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마는')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마르다')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불')&(first_tokens[i+1] == '출마')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '소')&(first_tokens[i+1] == '신')  :
                first_tokens[i] = '소신'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '신')&(first_tokens[i+1] == '문')  :
                first_tokens[i] = '신문'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '문')&(first_tokens[i+1] == '제인')  :
                first_tokens[i] = '문재인'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '문')&(first_tokens[i+1] == '갈이')  :
                first_tokens[i] = '물갈이'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마하')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마로')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마자')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '불출')&(first_tokens[i+1] == '마한')  :
                first_tokens[i] = '불출마'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '문')&(first_tokens[i+1] == '제')  :
                first_tokens[i] = '문제'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '상향')&(first_tokens[i+1] == '식')  :
                first_tokens[i] = '상향식'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '하향')&(first_tokens[i+1] == '식')  :
                first_tokens[i] = '하향식'
                first_tokens[i+1] = ' '
            elif (first_tokens[i] == '무성'):
                first_tokens[i] = '김무성'
            line_idx += 1
            if (line_idx % 50000) ==0 :
                print(str(line_idx),'번째 Token 재결합 ')

    # task3. 한 음절, 숫자는 filtering-out
    first_tokens = list(filter(lambda x : (len(x) != 1) & (x.isdecimal()==False), first_tokens))
    print('클렌징 후 토큰 수 : ',len(first_tokens))

    # task4. 불용어 지정 
    print('\n task4.  stopwords 제거') # <<< 위에서 stop_words 변수 지정 했습니다.
    first_tokens = [ind_token for ind_token in first_tokens if ind_token not in stop_words]
    print('stopwords 제거 후 Tokens :',len(first_tokens))
    print('ex) Tokens[:15] :',first_tokens[:15])

    # 토큰 전체를 nltk.text의 인스턴스로 지정 해줘서 
    text_object_name = Target_keyword + '_' + each_month
    whole_tokens_text = nltk.Text(first_tokens, name =text_object_name)
    # 빈도 선 그림을 그릴 때 nltk.Text(토큰)를 그대로 받아서 plotting을 하게 됩니다. 
    # 이때, 토큰은 > ['공천','공천','국회','안철수','안철수', ... '문재인', ...]
    # 이런 형태로 있고 이걸, nltk.Text(위에 토큰들) 로 넣어주면 => <NLTK. TEXT ...> 형태의 객체가 됩니다.

    #freq_rank_upto = 70 < 위에서 이렇게 지정하게 되고, 빈도가 높은 단어 70위 까지만 보관하게 됩니다.
    token_backup_1 = whole_tokens_text.vocab().most_common(freq_rank_upto) 
    token_list_1 = [token[0] for token in token_backup_1] # < 70개의 토큰
    freq_list_1 = [token[1] for token in token_backup_1] # < 70개의 토큰 별 counts 값

    # 꼭 포함해주어야 하는 필수 단어를 새 리스트 안에 넣고자 합니다. 
    appndx_word_list = []
    appndx_word_freq = []
    
    # task6 ) 빈도가 낮아도 빈도 카운트를 꼭 알아야하는 주요 단어를 포함시켜줍니다.
    for ind_token in necessary_words: # < 위에서 지정한 necessary_words를 하나씩 받아서 
        if ind_token not in token_list_1: # 상위 70위 단어 안에 없으면 
            appndx_word_list.append(ind_token) # 새로 넣어줍니다. 
            ind_freq=whole_tokens_text.count(ind_token)
            appndx_word_freq.append(ind_freq)

    token_list_1.extend(appndx_word_list)
    freq_list_1.extend(appndx_word_freq)
    print('필수 단어가 병합된 상위 토큰 수 :',len(token_list_1))
    
    # task 5) 빈도 테이블을 xlsx로 백업합니다.!
    backup_directory = '../' + Target_keyword + '_빈도_'+str(each_month)+'.xlsx'
    pd.DataFrame({'words' : token_list_1,
                  'freq':freq_list_1 }).to_excel(backup_directory, index = False)
    print(Target_keyword, '_빈도테이블_', str(each_month),'>> [빈도 테이블.xlsx]백업 완료')


공천 2016.01  빈도 분석 수행

3사 댓글 갯수 :  4162
===== 1차 Tokenization 수행 시작 =====
리플 수 = > 라인 수 :  4162
4000 번째 라인 pos 필터링
에러 라인 수:  0
조사, 어미, 문장부호 제거 후
리플 라인 예 =>  두당이 열심히 노력해서 빛나는 은메달 많이 따시기 바랍니다. ㅎㅎ  ===> pos 필터링 결과 예 => 두 당 열심히 노력 하다 빛나다 은메달 많이 따다 바라다 ㅎㅎ
pos 필터링 결과     =====> <Text>객체화 합니다..
pos 필터링 결과 예 :  두 당 열심히 노력 하다 빛나다 은메달 많이 따다 바라다 ㅎㅎ  ====> WholeText[0:20]... :  두 당 열심히 노력 하다 빛나다 은메
pos tagging 수행 =====>
1. 토큰화 :  토큰 갯수 =====> 121361
토큰 예시 : first_tokens[:10]... =>  ['두', '당', '열심히', '노력', '하다', '빛나다', '은메달', '많이', '따다', '바라다']
2. 클렌징 수행, 1. 해부된 단어 재결합 + 한 글자, 숫자 filtering-out
10000 번째 라인 재결합 작업
20000 번째 라인 재결합 작업
30000 번째 라인 재결합 작업
40000 번째 라인 재결합 작업
50000 번째 라인 재결합 작업
60000 번째 라인 재결합 작업
70000 번째 라인 재결합 작업
80000 번째 라인 재결합 작업
90000 번째 라인 재결합 작업
100000 번째 라인 재결합 작업
110000 번째 라인 재결합 작업
120000 번째 라인 재결합 작업
클렌징 후 토큰 수 :  92883
3.  stopwords 제거
3차 Tokens 갯수 : 73323
1차 선별된 상위 토큰 : ['국민', '정치', '사람', '김무성', '대통령', '안철수', '대표', '야당', '국회', '생각', '문재인', '공천', '의원', '새누리당', '지금', '여당', '국회의원'

## 추가 task 2)   
* concordance 체크 

In [2]:
text_object_name = Target_keyword + '_' + each_month # ex ) '공천_2019.10'
whole_tokens_text = nltk.Text(first_tokens, name = text_object_name)

NameError: name 'nltk' is not defined

In [26]:
print(whole_tokens_text.concordance("아들"))
print(whole_tokens_text.concordance("아"))
print(whole_tokens_text.concordance("들"))


Displaying 25 of 32 matches:
다 의원 들 줄 세우다 정말 가다 떨다 앉다 김무성 이 전라도 갑부 아들 태어나다 이 세상 보여주다 아무 것 없다 부자 사실 아니다 모르다 
 가볍다 뜻 크다 것 기대하다 틀리다 조선족 받아들이다 하다 김영삼 아들 늘다 이렇다 말 들 그 신뢰 하다 어렵다 하다 것 들 유 의원 하다
끼 학교 아부지 맘 고생 시키다 보다 ㅎㅎㅎ ㅋㅋㅋ 왜럼 헌병 오장 아들 0 답 다 그 리다 그려슈 당신 그 리다 부르다 짓다 친일파 진정 
 가지가지 하다 언제 다시 더 불당 복귀 하다 권력 재산 없다 사람 아들 구제 하다 위해 학교 찾다 가다 순수하다 부모 마음 되다 권력 있다
출 선봉 장 자칭 하다 때 그때 누구 걸리다 그러나 악질 친일 헌병 아들 임 밝혀지다 아들 로우 스쿨 외압 관련 오히려 피해자 그걸 믿다 사
하다 때 그때 누구 걸리다 그러나 악질 친일 헌병 아들 임 밝혀지다 아들 로우 스쿨 외압 관련 오히려 피해자 그걸 믿다 사람 있다 보다 모양
범 출신 고문 자행 일본헌병 군조 지금 중사 시 케미 시 구 니 오 아들 신기남 억울하다 담 출마 준비 하라 입 있다 하다 말 없다 비리 의
상임 명 심하다 뭐 장발장 정의 증말 소가 웃 일이 일본 순사 오장 아들 xxxxxx 야 국회의원 뱃지 빼다 그냥 조용하다 사라지다 보기 좋
사라지다 보기 좋다 본인 좋다 것 인간 욕망 측은하다 정의 왜 헌병 아들 로서 남 친일파 매도 하다 당시 그 단어 생각 안 나다 난 정의 같
신기남 아버지 일제시대 때 일본 압잡 이 국민 들 괴롭히다 이제 그 아들 신기남 로스쿨 청탁 의혹 로 로스쿨 학생 들 피 해주다 잘 하다 잘
 법더 민주당 탈당 자다 결정 이 다 역시 친일 앞잡이 헌 병오 장 아들 답 구나 어디 붙다 것 안철수 모시 다 않다 따다 불 민주당 국민 
기남 헌병 오장 이다 부친 띠 일본 망명 하다 하다 친일파 잔손 그 아들 당신 실제 억울 하다 수도 있다 문제 상대방 똑같이 억울 하다 수 
 넘다 구케 들어가다 구케 하수도 더 더럽

In [16]:
print(whole_tokens_text.concordance("세습"))

print(whole_tokens_text.concordance("세"))
print(whole_tokens_text.concordance("습"))

Displaying 8 of 8 matches:
 득세 하다 나라 없다 북 잘 되다 동포 잘 되다 것 아니다 3 대 세습 득세 인민 노예 되다 그리고 우리 영원하다 중국 일본 묻히다 말다 
다 더 민주 민주화 정의 무엇 자유민주 자유 빼 하다 전례 보아 북 세습 독재정 권 인민 민주주의 도 민주 포함 시키다 건가 자유 민주주의 
리 대한민국 국민 일치단결 힘 뿐 중구 난방 이소리 저 소리 3 대 세습 3 대다 뚱보 좋아하다 소리 과거 발언 행동 어떠하다 이제 대한민국
옛날 향수 젖다 분탕질 획책 하다 간 너희 교주 비참하다 최후 결과 세습 되다 것 알다 것비박 친박 갈다 수록 가관 박 대통령 마케팅 하다 
다 임 의원님 수고 많이 하다 이 젠 북 가다 사다 북한 개혁 하다 세습 왕조 끊다 내다 민주 혁명 북조선 일으키다 한국 하다 북 서다 하다
 한반도 미래 자신 들 권력 욕 앞세우다 때문 이북 김성주 3 대다 세습 독재 추종 하다 자 들 집단 구성 야당 행세 하다 있다 알다 최선 
세력 탈락 시키다 남다 자도 없다 김홍걸 공천 하 김대중 인민공화국 세습 하다 장해 먹다 김종인 좋아하다 욕 잘 하다 놈 들 포함 하다 절반
거 국민 들이다 바로 잡 라나 국제 테러 단체 이북 김성주 3 대다 세습 독재 추종 나라 뒤지다 엎 못 해 발악 하다 자 들 집단 구성 야당
None
Displaying 25 of 99 matches:
아니다 당 대표 로서 출전 하다 자기 당 후보 들 격려 하다 것 무슨 세 결합 통지 하다 대도 안 나오다 인사 들 다른 꿍꿍 이 있다 아니다 
OO 같다 패거리 정치 ㅡ 무대 배나무 밑 갓 끈 매다 말 속담 대하 세 삼 생각 하다 하다 예컨데 종북 떼법 정치 세력 언론 티끌 만 흠 잡
굴 마 담들다 섭외 하다 시간 없다 같다 참으로 기 막히다 어제 23 세 축구 보다 2:0 이기 다 기고 만장 하다 2:3 무참 히 깨지다 꼴
국민당 언론 크게 나오다 새누리당 조용하다 새롭다 능력자 들 영 입하 세 엄홍길 국회의원 하다 해도 자유 자질 되다 등산